# AGENT #

An agent, as defined in 2.1 is anything that can perceive its <b>environment</b> through sensors, and act upon that environment through actuators based on its <b>agent program</b>. This can be a dog, robot, or even you. As long as you can perceive the environment and act on it, you are an agent. This notebook will explain how to implement a simple agent, create an environment, and create a program that helps the agent act on the environment based on its percepts.

Before moving on, review the <b>Agent</b> and <b>Environment</b> classes in [<b>agents.py</b>](https://github.com/aimacode/aima-python/blob/master/agents.py).

Let's begin by importing all the functions from the agents.py module and creating our first agent - a blind dog.

In [1]:
from agents import *

class BlindDog(Agent):
    def eat(self, thing):
        print("Dog: Ate food at {}.".format(self.location))
            
    def drink(self, thing):
        print("Dog: Drank water at {}.".format( self.location))

dog = BlindDog()

What we have just done is create a dog who can only feel what's in his location (since he's blind), and can eat or drink. Let's see if he's alive...

In [2]:
print(dog.alive)

True


![Cool dog](https://gifgun.files.wordpress.com/2015/07/wpid-wp-1435860392895.gif)
This is our dog. How cool is he? Well, he's hungry and needs to go search for food. For him to do this, we need to give him a program. But before that, let's create a park for our dog to play in.

# ENVIRONMENT #

A park is an example of an environment because our dog can perceive and act upon it. The <b>Environment</b> class in agents.py is an abstract class, so we will have to create our own subclass from it before we can use it. The abstract class must contain the following methods:

<li><b>percept(self, agent)</b> - returns what the agent perceives</li>
<li><b>execute_action(self, agent, action)</b> - changes the state of the environment based on what the agent does.</li>

In [3]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


# PROGRAM - BlindDog #
Now that we have a <b>Park</b> Class, we need to implement a <b>program</b> module for our dog. A program controls how the dog acts upon it's environment. Our program will be very simple, and is shown in the table below.
<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>move down</td>
   </tr>
        
</table>


In [4]:
class BlindDog(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            #print("Dog: Ate food at {}.".format(self.location))
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            #print("Dog: Drank water at {}.".format(self.location))
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

Lets now run our simulation by creating a park with some food, water, and our dog.

In [5]:
park = Park()
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, 1)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

park.run(5)

BlindDog decided to move down at location: 1
BlindDog decided to move down at location: 2
BlindDog decided to move down at location: 3
BlindDog decided to move down at location: 4
BlindDog ate Food at location: 5


Notice that the dog moved from location 1 to 4, over 4 steps, and ate food at location 5 in the 5th step.

Lets continue this simulation for 5 more steps.

In [6]:
park.run(5)

BlindDog decided to move down at location: 5
BlindDog decided to move down at location: 6
BlindDog drank Water at location: 7


Perfect! Note how the simulation stopped after the dog drank the water - exhausting all the food and water ends our simulation, as we had defined before. Lets add some more water and see if our dog can reach it.

In [7]:
park.add_thing(water, 15)
park.run(10)

BlindDog decided to move down at location: 7
BlindDog decided to move down at location: 8
BlindDog decided to move down at location: 9
BlindDog decided to move down at location: 10
BlindDog decided to move down at location: 11
BlindDog decided to move down at location: 12
BlindDog decided to move down at location: 13
BlindDog decided to move down at location: 14
BlindDog drank Water at location: 15


This is how to implement an agent, its program, and environment. However, this was a very simple case. Lets try a 2-Dimentional environment now with multiple agents.


# 2D Environment #
To make our Park 2D, we will need to make it a subclass of <b>XYEnvironment</b> instead of Environment. Please note that our park is indexed in the 4th quadrant of the X-Y plane.

We will also eventually add a person to pet the dog.

In [8]:
class Park2D(XYEnvironment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

class BlindDog(Agent):
    location = [0,1]# change location to a 2d value
    direction = Direction("down")# variable to store the direction our dog is facing
    
    def movedown(self):
        self.location[1] += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

Now lets test this new park with our same dog, food and water

In [9]:
park = Park2D(5,20) # park width is set to 5, and height to 20
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,1])
park.add_thing(dogfood, [0,5])
park.add_thing(water, [0,7])
morewater = Water()
park.add_thing(morewater, [0,15])
park.run(20)

BlindDog decided to move down at location: [0, 1]
BlindDog decided to move down at location: [0, 2]
BlindDog decided to move down at location: [0, 3]
BlindDog decided to move down at location: [0, 4]
BlindDog ate Food at location: [0, 5]
BlindDog decided to move down at location: [0, 5]
BlindDog decided to move down at location: [0, 6]
BlindDog drank Water at location: [0, 7]
BlindDog decided to move down at location: [0, 7]
BlindDog decided to move down at location: [0, 8]
BlindDog decided to move down at location: [0, 9]
BlindDog decided to move down at location: [0, 10]
BlindDog decided to move down at location: [0, 11]
BlindDog decided to move down at location: [0, 12]
BlindDog decided to move down at location: [0, 13]
BlindDog decided to move down at location: [0, 14]
BlindDog drank Water at location: [0, 15]


This works, but our blind dog doesn't make any use of the 2 dimensional space available to him. Let's make our dog more energetic so that he turns and moves forward, instead of always moving down. We'll also need to make appropriate changes to our environment to be able to handle this extra motion.

# PROGRAM - EnergeticBlindDog #

Lets make our dog turn or move forwards at random - except when he's at the edge of our park - in which case we make him change his direction explicitly by turning to avoid trying to leave the park. Our dog is blind, however, so he wouldn't know which way to turn - he'd just have to try arbitrarily.

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>
       <table>
           <tr>
               <td><b>Remember being at Edge : </b></td>
               <td>At Edge</td>
               <td>Not at Edge</td>
           </tr>
           <tr>
               <td><b>Action : </b></td>
               <td>Turn Left / Turn Right <br> ( 50% - 50% chance )</td>
               <td>Turn Left / Turn Right / Move Forward <br> ( 25% - 25% - 50% chance )</td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

In [10]:
from random import choice

turn = False# global variable to remember to turn if our dog hits the boundary
class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (ie valid destination location)'''
        global turn
        if not success:
            turn = True # if edge has been reached, remember to turn
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            #print("Dog: Ate food at {}.".format(self.location))
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            #print("Dog: Drank water at {}.".format(self.location))
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    global turn
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    if turn: # then recall if you were at an edge and had to turn
        turn = False
        choice = random.choice((1,2));
    else:
        choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'
    

We also need to modify our park accordingly, in order to be able to handle all the new actions our dog wishes to execute. Additionally, we'll need to prevent our dog from moving to locations beyond our park boundary - it just isn't safe for blind dogs to be outside the park by themselves.

In [11]:
class Park2D(XYEnvironment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
            #print('now facing {}'.format(agent.direction.direction))
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
            #print('now facing {}'.format(agent.direction.direction))
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            #print('{} at {} facing {}'.format(agent, loc, agent.direction.direction))
            if self.is_inbounds(loc):# move only if the target is a valid location
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
            else:
                print('{} decided to move {}wards at location: {}, but couldnt'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward(False)
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


In [12]:
park = Park2D(3,3)
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [2,1])
morewater = Water()
park.add_thing(morewater, [0,2])
print('dog started at [0,0], facing down. Lets see if he found any food or water!')
park.run(20)

dog started at [0,0], facing down. Lets see if he found any food or water!
EnergeticBlindDog decided to move downwards at location: [0, 0]
EnergeticBlindDog decided to move downwards at location: [0, 1]
EnergeticBlindDog drank Water at location: [0, 2]
EnergeticBlindDog decided to turnright at location: [0, 2]
EnergeticBlindDog decided to move leftwards at location: [0, 2], but couldnt
EnergeticBlindDog decided to turnright at location: [0, 2]
EnergeticBlindDog decided to turnright at location: [0, 2]
EnergeticBlindDog decided to turnleft at location: [0, 2]
EnergeticBlindDog decided to turnleft at location: [0, 2]
EnergeticBlindDog decided to move leftwards at location: [0, 2], but couldnt
EnergeticBlindDog decided to turnleft at location: [0, 2]
EnergeticBlindDog decided to turnright at location: [0, 2]
EnergeticBlindDog decided to move leftwards at location: [0, 2], but couldnt
EnergeticBlindDog decided to turnleft at location: [0, 2]
EnergeticBlindDog decided to move downwards at l

This is good, but it still lacks graphics. What if we wanted to visualize our park as it changed? To do that, all we have to do is make our park a subclass of <b>GraphicEnvironment</b> instead of <b>XYEnvironment</b>. Lets see how this looks.

In [13]:
class GraphicPark(GraphicEnvironment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
            #print('now facing {}'.format(agent.direction.direction))
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
            #print('now facing {}'.format(agent.direction.direction))
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            #print('{} at {} facing {}'.format(agent, loc, agent.direction.direction))
            if self.is_inbounds(loc):# move only if the target is a valid location
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
            else:
                print('{} decided to move {}wards at location: {}, but couldnt'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward(False)
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


That is the only change we make. The rest of our code stays the same. There is a slight difference in usage though. Every time we create a GraphicPark, we need to define the colors of all the things we plan to put into the park. The colors are defined in typical [<b>RGB digital 8-bit format</b>](https://en.wikipedia.org/wiki/RGB_color_model#Numeric_representations), common across the web.

In [19]:
park = GraphicPark(5,5, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])
morewater = Water()
morefood = Food()
park.add_thing(morewater, [2,4])
park.add_thing(morefood, [4,3])
print('dog started at [0,0], facing down. Lets see if he found any food or water!')
park.run(20)

dog started at [0,0], facing down. Lets see if he found any food or water!


,,,,
,,,,
,,,,
,,,,
,,,,


EnergeticBlindDog decided to move downwards at location: [0, 0]


,,,,
,,,,
,,,,
,,,,
,,,,


EnergeticBlindDog drank Water at location: [0, 1]


...

,,,,
,,,,
,,,,
,,,,
,,,,


EnergeticBlindDog decided to turnright at location: [0, 3]


,,,,
,,,,
,,,,
,,,,
,,,,


EnergeticBlindDog decided to move upwards at location: [0, 3]


,,,,
,,,,
,,,,
,,,,
,,,,










## Wumpus Environment

In [4]:
from ipythonblocks import BlockGrid
from agents import *

color = {"Breeze": (225, 225, 225),
        "Pit": (0,0,0),
        "Gold": (253, 208, 23),
        "Glitter": (253, 208, 23),
        "Wumpus": (43, 27, 23),
        "Stench": (128, 128, 128),
        "Explorer": (0, 0, 255),
        "Wall": (44, 53, 57)
        }

def program(percepts):
    '''Returns an action based on it's percepts'''
    print(percepts)
    return input()

w = WumpusEnvironment(program, 7, 7)         
grid = BlockGrid(w.width, w.height, fill=(123, 234, 123))

def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]

def step():
    global grid, w
    draw_grid(w.get_world())
    grid.show()
    w.step()

In [5]:
step()

,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


[[<Bump>], [<Breeze>], [<Bump>], [<Breeze>], [<Breeze>, None]]
Forward
